In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import math
from cmdstanpy import CmdStanModel

In [3]:
# Prepare for the data
# 20,100,1000,5000
ibd = pd.read_json('../ibd_admix_data.json')
ibd['index'] = ibd['index'].apply(tuple)
ibd['group'] = ibd['group'].apply(tuple)
grouped = ibd.groupby(['group','u','v'])
n_boot = 1000

columns = ['group','mean_fraction','mean_number','boot_var','p_0','u','v']
boot_data = pd.DataFrame(columns = columns)
for name, group in grouped:
    gg = name[0]
    u = name[1]
    v = name[2]
    non_zero = [i for i in group['fraction'] if i > 0]
    p_0 = 1- (len(non_zero)/len(group['fraction']))
    number_mean = group['number'].mean()
    sample_mean = group['fraction'].mean()
    bootstrap_means = []

    for _ in range(n_boot):
        sample = np.random.choice(group['fraction'], size=len(group['fraction']), replace=True)
        bootstrap_means.append(np.mean(sample))

# Estimate variance
    variance_estimate = np.var(bootstrap_means)
    new_row = [gg,sample_mean,number_mean,variance_estimate,p_0,u,v]
    boot_data.loc[len(boot_data)] = new_row

In [4]:
print(boot_data.head())

    group  mean_fraction  mean_number      boot_var       p_0     u     v
0  (0, 0)       0.000863     0.238462  3.673405e-09  0.788462  0.70  0.75
1  (0, 0)       0.000836     0.215385  4.913267e-09  0.821795  0.75  0.80
2  (0, 0)       0.000965     0.234615  4.713845e-09  0.788462  0.80  0.85
3  (0, 0)       0.000801     0.183333  4.545750e-09  0.834615  0.85  0.90
4  (0, 0)       0.000647     0.139744  4.549864e-09  0.875641  0.90  0.95


In [5]:
boot_data['mean_fraction'] = boot_data['mean_fraction'] * 1000
boot_data['boot_var'] = boot_data['boot_var'] * 1000000
boot_data['group'] = boot_data['group'].apply(list)
json_ibd = boot_data.to_dict(orient='list')
print(json_ibd)
json_ibd['N_obs'] = len(boot_data)
json_ibd['length'] = 200
json_ibd['outgroup_N'] = 14000
json_ibd['ancestral_T'] = 400
json_ibd['T2'] = 150
json_ibd['ancestral_N'] = 14000


{'group': [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 2], [0, 2], [0, 2], [0, 2], [0, 2], [0, 2], [0, 2], [0, 2], [0, 2], [0, 2], [0, 2], [0, 2], [0, 2], [0, 2], [0, 2], [0, 2], [0, 2], [0, 2], [0, 2], [0, 3], [0, 3], [0, 3], [0, 3], [0, 3], [0, 3], [0, 3], [0, 3], [0, 3], [0, 3], [0, 3], [0, 3], [0, 3], [0, 3], [0, 3], [0, 3], [0, 3], [0, 3], [0, 3], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 3], [1, 3], [1, 3], [1, 3], [1, 3], [1, 3], [1, 3], [1, 3], [1, 3], [1, 3

In [6]:
boot_data

,group,mean_fraction,mean_number,boot_var,p_0,u,v
0,"[0, 0]",0.862884,0.238462,0.003673,0.788462,0.70,0.75
1,"[0, 0]",0.835594,0.215385,0.004913,0.821795,0.75,0.80
2,"[0, 0]",0.965471,0.234615,0.004714,0.788462,0.80,0.85
3,"[0, 0]",0.801456,0.183333,0.004546,0.834615,0.85,0.90
4,"[0, 0]",0.646661,0.139744,0.004550,0.875641,0.90,0.95
...,...,...,...,...,...,...,...
185,"[3, 3]",0.267201,0.010256,0.008705,0.989744,5.00,5.50
186,"[3, 3]",0.221627,0.007692,0.008044,0.992308,5.50,6.00
187,"[3, 3]",0.449353,0.014103,0.018869,0.985897,6.00,7.00
188,"[3, 3]",0.288031,0.007692,0.013377,0.992308,7.00,8.00


In [25]:
model = CmdStanModel(stan_file="ture_ibd_popn4_bootstrap.stan")

NameError: name 'CmdStanModel' is not defined

In [8]:
fit = model.variational(data = json_ibd, iter = 10000)


20:50:35 - cmdstanpy - INFO - Chain [1] start processing
20:50:35 - cmdstanpy - INFO - Chain [1] done processing


In [9]:
print(fit.variational_params_dict)


OrderedDict({'lp__': 0.0, 'log_p__': 0.0, 'log_g__': 0.0, 'N[1]': 5309.57, 'N[2]': 9608.12, 'N[3]': 5558.81, 'N[4]': 15393.8, 'T1': 27.7123, 'fraction': 0.256834})


In [10]:
fit = model.sample(data=json_ibd, chains=4, iter_sampling=1000, iter_warmup=500, seed=123)

20:50:45 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

20:51:11 - cmdstanpy - INFO - CmdStan done processing.


In [11]:
fit.summary()

,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
lp__,-51958.500000,0.127344,3.022860,-51961.900000,-51958.100000,-51956.200000,563.478000,16.106300,1.005410
N[1],5295.110000,0.215427,10.818400,5277.390000,5295.020000,5312.730000,2521.880000,72.084500,0.999256
N[2],9619.700000,8.933470,239.266000,9290.270000,9602.130000,9974.540000,717.333000,20.504000,1.006060
N[3],5631.650000,0.165013,8.362450,5617.520000,5631.560000,5645.670000,2568.200000,73.408600,0.999911
N[4],15569.500000,0.305218,16.768700,15542.700000,15569.300000,15596.700000,3018.420000,86.277500,0.999922
T1,28.005827,0.001501,0.075673,27.884100,28.005300,28.127600,2542.424575,72.671847,0.999896
fraction,0.258033,0.000011,0.000555,0.257095,0.258036,0.258938,2396.870860,68.511387,0.999457
